# Empirical Analysis of Sorting Algorithms

## **0. Importing Required Libraries**

In [158]:
#

## **1. Introduction**

## **2. Objective**

## **3 Implementing Sorting Algorithms**

In [159]:
test_arrays = [
    [],
    [1],
    [5, 3, 1, 4, 2],
    [1, 2, 3, 4, 5],
    [5, 4, 3, 2, 1],
    [2, 2, 2, 2, 2],
    [3, 1, 4, 1, 5, 9, 2],
]

In [160]:
def sort(arr, func, copy):
        if copy:
            array = arr.copy()
            func(array)
            return array
        else:
            return func(arr)

In [161]:
def test_sorting(arrays, func, copy=False):
    for arr in arrays:
        result = sort(arr, func, copy)
        print(f"O: {arr} \t S: {result}")

### **3.1 Quick Sort**

In [162]:
def quick_sort(arr):
    if len(arr) <= 1:
        return arr

    pivot = arr[len(arr) // 2]

    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]

    return quick_sort(left) + middle + quick_sort(right)

In [163]:
print("Testing Quick Sort")
test_sorting(test_arrays, quick_sort)

Testing Quick Sort
O: [] 	 S: []
O: [1] 	 S: [1]
O: [5, 3, 1, 4, 2] 	 S: [1, 2, 3, 4, 5]
O: [1, 2, 3, 4, 5] 	 S: [1, 2, 3, 4, 5]
O: [5, 4, 3, 2, 1] 	 S: [1, 2, 3, 4, 5]
O: [2, 2, 2, 2, 2] 	 S: [2, 2, 2, 2, 2]
O: [3, 1, 4, 1, 5, 9, 2] 	 S: [1, 1, 2, 3, 4, 5, 9]


### **3.2 Merge Sort**

In [164]:
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result

In [165]:
print("Testing Merge Sort")
test_sorting(test_arrays, merge_sort)

Testing Merge Sort
O: [] 	 S: []
O: [1] 	 S: [1]
O: [5, 3, 1, 4, 2] 	 S: [1, 2, 3, 4, 5]
O: [1, 2, 3, 4, 5] 	 S: [1, 2, 3, 4, 5]
O: [5, 4, 3, 2, 1] 	 S: [1, 2, 3, 4, 5]
O: [2, 2, 2, 2, 2] 	 S: [2, 2, 2, 2, 2]
O: [3, 1, 4, 1, 5, 9, 2] 	 S: [1, 1, 2, 3, 4, 5, 9]


### **3.3 Heap Sort**

In [166]:
def heapify(arr, n, i):
    largest = i
    l = 2 * i + 1
    r = 2 * i + 2
    if l < n and arr[l] > arr[largest]:
        largest = l
    if r < n and arr[r] > arr[largest]:
        largest = r
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest)

def heap_sort(arr):
    n = len(arr)
    for i in range(n // 2 - 1, -1, -1):
        heapify(arr, n, i)
    for i in range(n - 1, 0, -1):
        arr[0], arr[i] = arr[i], arr[0]
        heapify(arr, i, 0)

In [167]:
print("Testing Heap Sort")
test_sorting(test_arrays, heap_sort, True)

Testing Heap Sort
O: [] 	 S: []
O: [1] 	 S: [1]
O: [5, 3, 1, 4, 2] 	 S: [1, 2, 3, 4, 5]
O: [1, 2, 3, 4, 5] 	 S: [1, 2, 3, 4, 5]
O: [5, 4, 3, 2, 1] 	 S: [1, 2, 3, 4, 5]
O: [2, 2, 2, 2, 2] 	 S: [2, 2, 2, 2, 2]
O: [3, 1, 4, 1, 5, 9, 2] 	 S: [1, 1, 2, 3, 4, 5, 9]


### **3.4 Insertion Sort**

In [168]:
def insertion_sort(arr):
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key

In [169]:
print("Testing Insertion Sort")
test_sorting(test_arrays, insertion_sort, True)

Testing Insertion Sort
O: [] 	 S: []
O: [1] 	 S: [1]
O: [5, 3, 1, 4, 2] 	 S: [1, 2, 3, 4, 5]
O: [1, 2, 3, 4, 5] 	 S: [1, 2, 3, 4, 5]
O: [5, 4, 3, 2, 1] 	 S: [1, 2, 3, 4, 5]
O: [2, 2, 2, 2, 2] 	 S: [2, 2, 2, 2, 2]
O: [3, 1, 4, 1, 5, 9, 2] 	 S: [1, 1, 2, 3, 4, 5, 9]


## **4. Experimental Setup**

In [170]:
#

## **5. Results and Analysis**

## **6. Conclusion**

### **6.1 Key Findings**

### **6.2 Choosing the Optimal Sorting Algorithm**

### **6.3 Final Remarks**